This notebook presents a dummy example of a segmentation pipeline to show what we expect as a submission file.
We assume a pipeline where first the images are cropped to a smaller bounding box; then a 3D segmentation model takes as input images resampled at 1 $mm^3$ and that the input shape of the model is 
[batch_size, x_dim, y_dim, z_dim, channel]. The last dimension (channel) is 2 and represents the CT and PT images.
The output image is a binary segmentation whose shape is [batch_size, x_dim, y_dim, z_dim, 1].
The final submissions must be expressed in the CT spatial domain (same spacing, origin, size, and direction).

In [ ]:
from pathlib import Path
import shutil

import numpy as np
import pandas as pd
import SimpleITK as sitk
from tqdm import tqdm

In [ ]:
image_folder = Path("../data/hecktor2022_testing/imageTs").resolve()
results_folder = Path("../data/dummy_segmentation_results/").resolve()
results_folder.mkdir(exist_ok=True)

In [ ]:
def dummy_model(x):
    return np.random.uniform(size=x.shape[:4] + (1, )) * 2
    
def dummy_cropper(im):
    delta_x = np.random.randint(0,100)
    delta_y = np.random.randint(0,100)
    delta_z = np.random.randint(0,100)
    x1 = im.GetOrigin()[0] + delta_x
    y1 = im.GetOrigin()[1] + delta_y
    z1 = im.GetOrigin()[2] + delta_z
    x2 = im.GetOrigin()[0] + delta_x + 144
    y2 = im.GetOrigin()[1] + delta_y + 144
    z2 = im.GetOrigin()[2] + delta_z + 144
    return np.asarray([x1,y1,z1,x2,y2,z2])

In [ ]:
patient_list = [f.name.split("__")[0] for f in image_folder.rglob("*__CT.nii.gz")]

In [ ]:
# Instantiating the resampler
resampling_spacing = np.array([1.0, 1.0, 1.0])
pre_resampler = sitk.ResampleImageFilter()
pre_resampler.SetInterpolator(sitk.sitkBSpline)
pre_resampler.SetOutputSpacing(resampling_spacing)

post_resampler = sitk.ResampleImageFilter()
post_resampler.SetInterpolator(sitk.sitkNearestNeighbor)

In [ ]:
for patient_id in tqdm(patient_list):
    # loading the images and storing the ct spacing
    image_ct = sitk.ReadImage(str(image_folder / (patient_id + "__CT.nii.gz")))
    image_pt = sitk.ReadImage(str(image_folder / (patient_id + "__PT.nii.gz")))
    spacing_ct = image_ct.GetSpacing()
    size_ct = image_ct.GetSize()
    direction_ct = image_ct.GetDirection()
    origin_ct = image_ct.GetOrigin()



    # Find the cropping of your images for model training 
    # (We assume a bounding box [x1,y1, z1, x2, y2, z2])
    bb = dummy_cropper(image_ct)

    # resampling the images
    resampled_size = np.round(
        (bb[3:] - bb[:3]) / resampling_spacing).astype(int)
    pre_resampler.SetOutputOrigin(bb[:3])
    pre_resampler.SetSize([int(k)
                           for k in resampled_size])  # sitk requires this
    image_ct = pre_resampler.Execute(image_ct)
    image_pt = pre_resampler.Execute(image_pt)

    # sitk to numpy, sitk stores images with [dim_z, dim_y, dim_x]
    array_ct = np.transpose(sitk.GetArrayFromImage(image_ct), (2, 1, 0))
    array_pt = np.transpose(sitk.GetArrayFromImage(image_pt), (2, 1, 0))

    # ... apply your preprocessing here

    x = np.stack([array_ct, array_pt], axis=-1)
    x = x[np.newaxis, ...]  # adding batch dimension
    segmentation = dummy_model(x)[0, :, :, :, 0]

    # do not forget to threshold your output (0: background, 1: GTVp, 2:GTVn)
    segmentation = np.round(segmentation).astype(np.uint8)

    # numpy to sitk
    image_segmentation = sitk.GetImageFromArray(
        np.transpose(segmentation, (2, 1, 0)))

    # Important: Set the correct origin and spacing 
    image_segmentation.SetOrigin(bb[:3])
    image_segmentation.SetSpacing(resampling_spacing)

    # Resample to the original CT resolution.
    # You are welcomed to use any fancier interpolation here.
    post_resampler.SetSize(size_ct)
    post_resampler.SetOutputDirection(direction_ct)
    post_resampler.SetOutputOrigin(origin_ct)
    post_resampler.SetOutputSpacing(spacing_ct)
    image_segmentation = post_resampler.Execute(image_segmentation) 


    image_segmentation = post_resampler.Execute(image_segmentation)

    # Saving the prediction
    sitk.WriteImage(
        image_segmentation,
        str(results_folder / (patient_id + ".nii.gz")),
    )


In [ ]:
# zip the folder and it is ready for submission
shutil.make_archive("../data/dummy_segmentation_submission", "zip", results_folder)